In [ ]:
from IPython import display
import argparse
import cv2
import gym
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys, shutil
import torch
import omegaconf
import time

import mbrl.env.cartpole_continuous as cartpole_env
import mbrl.env.reward_fns as reward_fns
import mbrl.env.termination_fns as termination_fns
import mbrl.models as models
import mbrl.planning as planning
import mbrl.util.common as common_util
import mbrl.util as util

import tactile_gym.rl_envs
from tactile_gym.sb3_helpers.params import import_parameters
from mbrl.util.plot_and_save_push_data import plot_and_save_push_plots

from pyvirtualdisplay import Display
from tactile_gym.sb3_helpers.params import import_parameters
from tactile_gym.rl_envs.nonprehensile_manipulation.object_push.object_push_env import get_states_from_obs

_display = Display(visible=False, size=(1400, 900))
_ = _display.start()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


In [ ]:
    # model_filename = 'training_model'
    # model_number = 50
    # work_dir = os.path.join(os.getcwd(), model_filename)
    work_dir = r"/home/qt21590/Documents/Projects/tactile_gym_mbrl/training_model/stochastic_icem_H25"
    model_dir = os.path.join(work_dir, 'model_trial_{}'.format(model_number))
    evaluation_result_directory = os.path.join(work_dir, "evaluation_result_model_{}".format(model_number))

    # work_dir = r"/home/qt21590/Documents/Projects/tactile_gym_mbrl/training_model/random_goal_update_orn_john_guide_off"
    # model_dir = work_dir
    # evaluation_result_directory = os.path.join(work_dir, "evaluation_result")

    if not os.path.exists(evaluation_result_directory):
        os.mkdir(evaluation_result_directory)
    else:
        for filename in os.listdir(evaluation_result_directory):
            file_path = os.path.join(evaluation_result_directory, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('Failed to delete %s. Reason: %s' % (file_path, e))

    # Load the environment 
    env_name = 'object_push-v0'
    env_kwargs_file = 'env_kwargs'
    env_kwargs_dir = os.path.join(work_dir, env_kwargs_file)
    env_kwargs = omegaconf.OmegaConf.load(env_kwargs_dir)
    env_kwargs["env_modes"]['eval_mode'] = True
    env_kwargs["env_modes"]['eval_num'] = num_test_trials

    env = gym.make(env_name, **env_kwargs)
    seed = 0
    env.seed(seed)
    rng = np.random.default_rng(seed=0)
    generator = torch.Generator(device=device)
    generator.manual_seed(seed)
    obs_shape = env.observation_space.shape
    act_shape = env.action_space.shape
